## First Weld Model of Detection

Til the date of this push in GIT, the suggestion is run de code in Colab GPU T4 to reason of computacional cost.

* Download a dataset, prepare it, train it, and save it.

### 1) Load Dataset 🤓

We will use a set of images of Industrial Welding Defects provided by the client.

In [ ]:
!unrar x "Weld_Defects.rar"

### 2) Dependencias 👌

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

### 3) Entrenamiento 💪

Cargar un modelo preentrenado

In [ ]:
model = YOLO("yolo11n.pt")

In [ ]:
yaml_path = "/content/defectos_de_soldaduras/data.yaml"

#### 📍 Opción 1: Configuración por default

In [ ]:
model.train(
    data=yaml_path,
    epochs=10,
    imgsz=640,
    augment=True,
    )

#### 📍 Opción 2) Patience con Early Stopping

Si no mejora en 5 epochs el loss validation, detiene el entrenamiento.

In [ ]:
model.train(
    data=yaml_path,
    epochs=20,
    imgsz=640,
    augment=True,
    patience=5,
    )

#### 📍 Opción 3) Custom Data Augmentation

Diferentes operaciones de Data Augmentation.

In [ ]:
model.train(
    data=yaml_path,
    epochs=10,
    imgsz=640,
    augment=True,
    patience=5,
    hsv_h=0.015,    # Ajuste de tono HSV (cambiar color)
    hsv_s=0.7,      # Ajuste de saturación HSV
    hsv_v=0.4,      # Ajuste de brillo HSV
    flipud=0.5,     # Volteo vertical con 50% de probabilidad
    fliplr=0.5,     # Volteo horizontal con 50% de probabilidad
    mosaic=1.0,     # Habilitar mosaico (mezcla de imágenes)
    mixup=0.5,      # Mezcla dos imágenes con probabilidad 50%
    perspective=0.0005,  # Transformación en perspectiva
    degrees=5.0,    # Rotación de ±5 grados
    shear=2.0,      # Inclinación de la imagen
    scale=0.5,      # Zoom en la imagen
)

### 4) Guardar el modelo 💾

Guarda el modelo completo (arquitectura + pesos)

In [ ]:
model.save("mi_modelo.pt")

Luego, para cargarlo:

In [ ]:
mi_modelo = YOLO("mi_modelo.pt")

### 5) Hacer Predicciones en Producción 🤙

#### 📍 1) Visualizar resultados

In [ ]:
import os
from IPython.display import Image, display

Crear carpeta para guardar resultados de prueba

In [ ]:
save_dir = "./testing/"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
image_path = "/content/defectos_de_soldaduras/valid/images/SampleV1_1_mp4-24_jpg.rf.8487d87bb4c1d1ab9059da84ac881495.jpg"

results = mi_modelo(image_path)

for result in results:
  result.plot(save=True, filename=os.path.join(save_dir, os.path.basename(result.path)))
  result_image_path = os.path.join(save_dir, os.path.basename(result.path))
  display(Image(filename=result_image_path))

#### 📍 2) Filtrar respuesta

Devolver solo los que tenga un confidence mayor a 30%.

In [ ]:
image_path = "/content/defectos_de_soldaduras/valid/images/SampleV1_1_mp4-24_jpg.rf.8487d87bb4c1d1ab9059da84ac881495.jpg"

results = mi_modelo(
    image_path,
    conf=0.30,
    )

for result in results:
  result.plot(save=True, filename=os.path.join(save_dir, os.path.basename(result.path)))
  result_image_path = os.path.join(save_dir, os.path.basename(result.path))
  display(Image(filename=result_image_path))

Devolver solo los que tenga un confidence mayor a 30% y sea de la clase "Good Weld".

In [ ]:
image_path = "/content/defectos_de_soldaduras/valid/images/SampleV1_1_mp4-24_jpg.rf.8487d87bb4c1d1ab9059da84ac881495.jpg"


results = mi_modelo(
    image_path,
    conf=0.30,
    classes=[1],
    )

for result in results:
  result.plot(save=True, filename=os.path.join(save_dir, os.path.basename(result.path)))
  result_image_path = os.path.join(save_dir, os.path.basename(result.path))
  display(Image(filename=result_image_path))

#### 📍 3) Devolver resultados específicos

In [ ]:
def detectar_objetos(image_path, model):
    """
    Realiza la detección de objetos en una imagen y devuelve:
      - Las cajas delimitadoras (bounding boxes) en formato [x1, y1, x2, y2].
      - Los nombres de las clases detectadas.
      - Los confidences de cada detección.
      - La imagen anotada con las detecciones.

    Parámetros:
      image_path (str): Ruta a la imagen.
      model: Modelo YOLO entrenado.

    Retorna:
      boxes (numpy.ndarray): Arreglo de bounding boxes.
      class_names (list): Lista con los nombres de las clases detectadas.
      confidences (numpy.ndarray): Arreglo con los scores/confidencias.
      annotated_img (numpy.ndarray): Imagen con las detecciones dibujadas.
    """
    # Realiza la detección (para una imagen, results es una lista y tomamos el primer resultado)
    results = model(image_path)
    result = results[0]

    # Extrae las cajas delimitadoras (bounding boxes) en formato xyxy
    boxes = result.boxes.xyxy.cpu().numpy()

    # Extrae los índices de las clases detectadas y las confidencias
    classes_idx = result.boxes.cls.cpu().numpy().astype(int)
    confidences = result.boxes.conf.cpu().numpy()

    # Si el modelo tiene el diccionario de nombres, se obtienen los nombres de las clases
    if hasattr(model, "names"):
        class_names = [model.names[i] for i in classes_idx]
    else:
        class_names = classes_idx  # Si no existe, se retorna los índices de clase

    # Se obtiene la imagen con los bounding boxes ya dibujados
    annotated_img = result.plot()

    return boxes, class_names, confidences, annotated_img

In [ ]:
image_path = "/content/defectos_de_soldaduras/valid/images/SampleV2_1_mp4-26_jpg.rf.a8431cb1acce1c55d32861c982c2a16a.jpg"

boxes, classes, confs, img_anotada = detectar_objetos(image_path, mi_modelo)

In [ ]:
img_anotada

In [ ]:
print("Bounding Boxes:", boxes)
print("Class Names:", classes)
print("Confidences:", confs)

### 6) Conclusiones

- Aprender sobre los distintos objetos y métodos que nos ofrece YOLO.

- Realizar el proceso completo de detección de imágenes con YOLO.

<br>
<br>
<br>

---

<br>
<br>


<img src="https://static.platzi.com/media/avatars/platziteam_8cfe6fc7-1246-4c9a-9f5d-d10d467443ee.png" width="100px">

[Platzi](https://platzi.com/) 🚀

